In [2]:
import numpy as np
import os
import pandas as pd
import SimpleFunctions as SF
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
# Grid
NPoints = 11
Nus     = np.linspace(0.395,0.455,NPoints+1)
Mus     = np.linspace(0.2,0.8,NPoints+1)
Alphas  = np.linspace(-30,-20,NPoints+1)


# Optimization Parameters
ConstitutiveModels = ['Ogden', 'Neo-Hookean']
ConstitutiveModel = ConstitutiveModels[0]

BCsTypes = ['Ideal', 'Fixed']
BCsType  = BCsTypes[1]

NumberElementsTested = [1, 2, 3, 5, 10, 20]
NumberElements = NumberElementsTested[5]

LoadCases = ['Compression', 'Tension', 'SimpleShear']
RelativeWeights = [1,1,1]


# Simulation parameters
FinalRelativeStretch = 0.1
RelativeStepSize     = 0.02


# Folder for the results
Output_Path = os.path.join('CostFunctionShape', BCsType, ConstitutiveModel)
os.makedirs(Output_Path, exist_ok=True)

FileName = str(NumberElements) + 'Elements' + str(NPoints) + '³Points.csv'

# Data frame to store values
Columns = ['Nu','Mu', 'Alpha', 'TotalCost']
DataFrame = pd.DataFrame(columns=Columns, index=range(int((NPoints+1)**3)), dtype='float64')

IterationTime = []

# Compute the map
Point = 0
for Nu in range(NPoints+1):
    for Mu in range(NPoints+1):        
        for Alpha in range(NPoints+1):
            
            # Initialize time tracking
            start = time.time()
            
            Parameters = [Nus[Nu], Mus[Mu], Alphas[Alpha]]
            
            TotalCost = SF.CostFunction(Parameters, ConstitutiveModel, BCsType, NumberElements, LoadCases, RelativeWeights, FinalRelativeStretch, RelativeStepSize, Plot = False)
            
            # Save values to table
            Parameters.append(TotalCost)
            DataFrame.loc[Point] = Parameters
            Point += 1
            
            IterationTime.append(time.time() - start)

# Save table
DataFrame.to_csv(os.path.join(Output_Path, FileName), index=False, sep=',')

In [ ]:
IterationTimes = np.array([159.205, 169.551])
print('Compilation time: %1.3f'%(IterationTimes[1]-IterationTimes[0]))
IterationTimes = np.array([159.205, 169.551, 168.555, 166.909, 165.516])
print('Mean Iteration time: %1.3f'%np.mean(IterationTimes))
IterationTime = np.array([159.205-158.716, 159.205-157.264, 159.205-155.129])
print('Mean time for new mesh generation: %1.3f'%np.mean(IterationTime))
a = (NPoints+1)**3*np.mean(IterationTimes)/(3600*24)
print('TotalTimeEstimated without new mesh %1.3f'%(a))
b = ((NPoints+1)**3)*(np.mean(IterationTimes)+np.mean(IterationTime))/(3600*24)
print('TotalTimeEstimated with new mesh %1.3f'%(b))

In [41]:
NPoints = 11

# Optimization Parameters
ConstitutiveModels = ['Ogden', 'Neo-Hookean']
ConstitutiveModel = ConstitutiveModels[0]

BCsTypes = ['Ideal', 'Fixed']
BCsType  = BCsTypes[1]


NumberElementsTested = [1, 2, 3, 5, 10, 20]
NumberElements = NumberElementsTested[5]

# Folder for the results
Output_Path = os.path.join('CostFunctionShape', BCsType, ConstitutiveModel)
Optim_Path = os.path.join('OptimizationResults', BCsType, ConstitutiveModel)

os.makedirs(Output_Path, exist_ok=True)

FileName = str(NumberElements) + 'Elements' + str(NPoints) + '³Points.csv'
OptiName = str(NumberElements) + 'Elements.txt'
MarcName = str(NumberElements) + 'Elements-M.txt'

# Cost Function Data
CostData = pd.read_csv(os.path.join(Output_Path, FileName), sep=',', decimal='.')
OptiData = pd.read_csv(os.path.join(Optim_Path, OptiName), sep=' ', decimal='.')
MarcData = pd.read_csv(os.path.join(Optim_Path, MarcName), sep=' ', decimal='.')

# Row with the minimum cost
print('Min cost point for:')
print(CostData.loc[CostData['TotalCost'].idxmin()])

Min cost point for:
Nu            0.400455
Mu            0.636364
Alpha       -22.727273
TotalCost     0.067716
Name: 248, dtype: float64


In [39]:
CostData = pd.read_csv('Results.csv', sep=',', decimal='.')
CostData

,Nu,Mu,Alpha,Cost
0,0.345,0.6,-10.0,0.472417
1,0.345,0.8,-10.0,0.323291
2,0.395,0.3,-60.0,38.009093
3,0.295,0.1,-22.5,1.311030
4,0.295,0.3,-10.0,0.931168
...,...,...,...,...
120,0.395,1.1,-10.0,0.421137
121,0.495,0.1,-60.0,NaN
122,0.445,0.6,-60.0,130.507430
123,0.445,0.8,-60.0,274.632697


In [40]:
# 3D Plot
%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

Nus    = CostData.Nu
Mus    = CostData.Mu
Alphas = CostData.Alpha
Costs  = CostData.Cost

DM, MuM, AlphaM = 1.026, 0.458, -59.367
NuM = (3-DM*MuM)/(DM*MuM+6)
Marcdf = pd.DataFrame([NuM, MuM, AlphaM])

Threshold = 1E-0
Subsampling = 1

Vmin = min(Costs)
Vmax = Threshold * (max(Costs) - min(Costs))

#img = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.jet(), vmin=0, vmax=0.1)
CostImage = ax.scatter(Nus[0::Subsampling], Mus[0::Subsampling], Alphas[0::Subsampling], c=Costs[0::Subsampling], cmap=plt.gray(), marker = '.', vmin=Vmin, vmax=Vmax, label='Points tested')
ColorBar=plt.colorbar(CostImage)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
#ax.plot([Nus[CostData['TotalCost'].idxmin()]], [Mus[CostData['TotalCost'].idxmin()]], [Alphas[CostData['TotalCost'].idxmin()]], marker = 'o', color='k', label='Grid min cost point')
ax.plot([OptiData.Nu[0]], [OptiData.Mu[0]], [OptiData.Alpha[0]], marker = 'o', color='m', label='Budday starting point')
ax.plot(OptiData.Nu, OptiData.Mu, OptiData.Alpha, marker = None, color='m', label='Optimization path')
ax.plot([OptiData.Nu[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], [OptiData.Alpha[len(OptiData)-1]], marker = 'o', color='r', label='Optimization result')
ax.plot([MarcData.Nu[0]], [MarcData.Mu[0]], [MarcData.Alpha[0]], marker = 'o', color='c', label='Marc starting point')
ax.plot(MarcData.Nu, MarcData.Mu, MarcData.Alpha, marker = None, color='c', label='Optimization path')
ax.plot([MarcData.Nu[len(MarcData)-1]], [MarcData.Mu[len(MarcData)-1]], [MarcData.Alpha[len(MarcData)-1]], marker = 'o', color='b', label='Optimization result')
ax.plot(Marcdf.loc[0], Marcdf.loc[1], Marcdf.loc[2], marker = 'o', color='g', label='Marc optimization result')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
# Filtering

MaxRow = CostData['TotalCost'].idxmax()

Threshold = 3.75E-2
MaxCost = max(CostData.TotalCost)

FilteredCost = pd.DataFrame(columns=CostData.columns)

for Point in range(len(CostData)):
    CostValue = CostData.TotalCost[Point]
    if CostValue < MaxCost * Threshold:
        FilteredCost = FilteredCost.append(CostData.loc[Point])
        
# Filtered 3D Plot
%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

Nus    = FilteredCost.Nu
Mus    = FilteredCost.Mu
Alphas = FilteredCost.Alpha
Costs  = FilteredCost.TotalCost

#img = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.jet(), vmin=0, vmax=0.1)
CostImage = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.viridis(), marker = '.', label='Points tested')
ColorBar=plt.colorbar(CostImage)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.plot([Nus[FilteredCost['TotalCost'].idxmin()]], [Mus[FilteredCost['TotalCost'].idxmin()]], [Alphas[FilteredCost['TotalCost'].idxmin()]], marker = 'o', color='k', label='Grid min cost point')
ax.plot([OptiData.Nu[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], [OptiData.Alpha[len(OptiData)-1]], marker = 'o', color='r', label='Optimization result')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
# 2D plot
%matplotlib widget

Xaxes = ['Nu', 'Mu', 'Alpha']
Yaxes = ['Nu', 'Mu', 'Alpha']


# Plot tuning
Xaxis = Xaxes[0]
Yaxis = Yaxes[1]
Threshold = 7E-2


# Plot
Vmin = min(CostData.TotalCost)
Vmax = Threshold * (max(CostData.TotalCost) - min(CostData.TotalCost))

MinPoint = CostData['TotalCost'].idxmin()

CostData.plot.scatter(x=Xaxis,y=Yaxis,c='TotalCost', colormap='viridis', vmin=Vmin, vmax=Vmax, marker = '.')
plt.plot(CostData[Xaxis][MinPoint], CostData[Yaxis][MinPoint], marker = 'o', color='k', markersize=10, fillstyle='none', label='Grid min cost point')
plt.plot([OptiData[Xaxis][len(OptiData)-1]], [OptiData[Yaxis][len(OptiData)-1]], marker = 'o', color='r', markersize=10, fillstyle='none', label='Optimization result')
plt.legend(loc='lower right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
%matplotlib widget

CostData.Nu
NuContour    = np.array(CostData.Nu[0:12**3:12**2])
MuContour    = np.array(CostData.Mu[0:12**2:12**1])
AlphaContour = np.array(CostData.Alpha[0:12**1:12**0])

NuMuCost    = np.array(CostData.TotalCost[int(np.argwhere(AlphaContour == CostData.Alpha[MinPoint])):12**3:12**1])

plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111)
CostContour = plt.contour(NuContour, MuContour, NuMuCost.reshape(12,12), np.linspace(min(NuMuCost), max(NuMuCost), 50))
ColorBar=plt.colorbar(CostContour)
ColorBar.set_label('Cost (-)')
plt.plot(CostData.Nu[MinPoint], CostData.Mu[MinPoint], marker = 'o', color='k', markersize=10, label='Grid min cost point')
plt.plot([OptiData.Nu[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], marker = 'o', color='r', markersize=10, label='Optimization result')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
plt.legend(loc='lower right', framealpha=1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
Values = pd.read_csv('Ranges.txt', sep=',', decimal='.')
Values

,Nu,Mu,Alpha


In [49]:
Values = pd.read_csv('Ranges.txt', sep=',', decimal='.')
range(len(Values))

range(0, 12128)

In [54]:
MuVal = [Values.loc[Iterations[0]].Mu, Values.loc[Iterations[0]+1].Mu]
MuVal

[0.1, 0.325]

In [51]:
Values.loc[Iterations[0]]

Nu         0.295
 Mu        0.100
 Alpha   -60.000
Name: 0, dtype: float64

In [104]:
NPoints = 11
Nus = np.linspace(0.295,0.495,NPoints)
Mus = np.linspace(0.1,1.1,NPoints)
Alphas = np.linspace(-60,-10,NPoints)

Nus = np.round(Nus,3)
Mus = np.round(Mus,3)
Alphas = np.round(Alphas,3)


print(Nus)
print(Mus)
print(Alphas)

[0.295 0.315 0.335 0.355 0.375 0.395 0.415 0.435 0.455 0.475 0.495]
[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1]
[-60. -55. -50. -45. -40. -35. -30. -25. -20. -15. -10.]


In [106]:
for Nu in range(NPoints-1):
    for Mu in range(NPoints-1):
        for Alpha in range(NPoints-1):

            Range  = np.array([Nus[Nu], Mus[Mu], Alphas[Alpha],Nus[Nu+1], Mus[Mu+1], Alphas[Alpha+1]])
            Range  = np.reshape(Range,(2,3))
            Range1 = {'Nu':Range[0,0], 'Mu':Range[0,1], 'Alpha':Range[0,2]}
            Range2 = {'Nu':Range[1,0], 'Mu':Range[1,1], 'Alpha':Range[1,2]}

            Values = Values.append(Range1, ignore_index=True)
            Values = Values.append(Range2, ignore_index=True)

Values.to_csv('Ranges.txt', index=False)

In [5]:
a = pd.read_pickle('results.pkl')
a.cost

1.9225623117742374